# Algo versus Doc Analysis LUMC
This notebook runs the analysis for the `algo_vs_doc` project for the LUMC hospital.

In [ ]:
%load_ext autoreload
%autoreload 2

# Load required packages
import os
import sys

# Define project root directory
project_root = '../'
# Add project root to system path to allow imports from project modules
sys.path.append(project_root)

import algo_vs_doc.plots as plots
import algo_vs_doc.analysis as analysis

import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency, contingency, ttest_ind, f_oneway, kruskal, alpha

pd.set_option("mode.chained_assignment", None)
pd.set_option("display.max_columns", None)

# Settings and Data

In [ ]:
# set current hospital for storing outputs and hospital specific data processing
hospital = "LUMC"
plots.hospital = hospital
analysis.hospital = hospital

data_folder = "data"
tables_folder = "tables"

predictions_and_outcomes = pd.read_csv(os.path.join(project_root, data_folder, "merged_data_lumc.tsv"), delimiter="\t")

## predictions_and_outcomes
- contains 245 rows of LUMC data
- each row corresponds to one prediction of model and doc
- contains the outcome (readmision or death)

In [ ]:
# Transform data
predictions_and_outcomes.loc[:, ["doc_prediction"]] = (
    predictions_and_outcomes.loc[:, ["doc_prediction"]] / 100
)
predictions_and_outcomes.loc[:, ["doc_prediction_cat"]] = (
    predictions_and_outcomes.loc[:, ["doc_prediction_cat"]] / 3
)

# recode confidence column to same categories as VUmc
predictions_and_outcomes = predictions_and_outcomes.rename(columns={'confidence': 'confidence_int'})
predictions_and_outcomes.loc[
    (predictions_and_outcomes.confidence_int == 1)
    | (predictions_and_outcomes.confidence_int == 2),
    "confidence",
] = "Gemiddeld/Weinig"
predictions_and_outcomes.loc[predictions_and_outcomes.confidence_int == 3, "confidence"] = (
    "Veel"
)
predictions_and_outcomes["true_h_doc"] = (
    pd.to_datetime(predictions_and_outcomes.discharge_time)
    - pd.to_datetime(predictions_and_outcomes.time_doc_prediction)
).apply(lambda d: d.seconds / 3600)

# at lumc there is no difference between algo time and discharge time, as we score the predictions at the discharge moment
predictions_and_outcomes["true_h_algo_at_discharge"] = 0

predictions_and_outcomes["true_h_algo_at_doc_prediction"] = (
    pd.to_datetime(predictions_and_outcomes.discharge_time)
    - pd.to_datetime(predictions_and_outcomes.time_doc_prediction)
).apply(lambda d: d.seconds / 3600)

# Exploratory data analysis


## Plots

In [ ]:
# scipy dependency should be > 1.10.0 to use 'count' attribute from res CrossTabResult

contingency_table_data = predictions_and_outcomes.loc[
    ~pd.isnull(predictions_and_outcomes.doc_prediction_cat), :
]
contingency_table = contingency.crosstab(
    contingency_table_data.doc_prediction_cat, contingency_table_data.outcome
).count
chi2 = chi2_contingency(contingency_table)
plots.barplot_by_outcome(data=predictions_and_outcomes, variable="doc_prediction_cat", chi2=chi2)

In [ ]:
ttest = [
    ttest_ind(
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 0, "doc_prediction"
        ],
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 1, "doc_prediction"
        ],
        nan_policy="omit",
    ),
    ttest_ind(
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 0, "algo_at_doc_prediction"
        ],
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 1, "algo_at_doc_prediction"
        ],
        nan_policy="omit",
    ),
    ttest_ind(
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 0, "algo_at_discharge"
        ],
        predictions_and_outcomes.loc[
            predictions_and_outcomes.outcome == 1, "algo_at_discharge"
        ],
        nan_policy="omit",
    ),
]
plots.histogram_by_outcome(
    data=predictions_and_outcomes,
    variables=["doc_prediction", "algo_at_doc_prediction", "algo_at_discharge"],
    ttest=ttest,
)

In [ ]:
anova = f_oneway(
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 1 / 3, "doc_prediction"
    ],
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 2 / 3, "doc_prediction"
    ],
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 1, "doc_prediction"
    ],
)
kruskal_wallis = kruskal(
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 1 / 3, "doc_prediction"
    ],
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 2 / 3, "doc_prediction"
    ],
    predictions_and_outcomes.loc[
        predictions_and_outcomes["doc_prediction_cat"] == 1, "doc_prediction"
    ],
)
plots.stratified_histogram(
    predictions_and_outcomes,
    "doc_prediction",
    "doc_prediction_cat",
    kruskal_wallis,
    labels=["Low Risk", "Moderate Risk", "High Risk"],
    plotname="physician_model_prediction",
    title="Physician Predictions by Risk category"
)

## Correlation

In [ ]:
correlations = predictions_and_outcomes[
    ["algo_at_doc_prediction", "algo_at_discharge", "doc_prediction"]
    ].corr("spearman")
correlations.to_csv(os.path.join(project_root, tables_folder, hospital, "correlation_table.csv"))
plots.scatter_plot(
    data=predictions_and_outcomes,
    xs=["doc_prediction", "doc_prediction", "algo_at_doc_prediction"],
    ys=["algo_at_doc_prediction", "algo_at_discharge", "algo_at_discharge"],
    correlation=[
        correlations.iloc[0, 2],
        correlations.iloc[1, 2],
        correlations.iloc[1, 0],
    ],
)

## Calculate the combined prediction of physicians and model

In [ ]:
predictions_and_outcomes = analysis.combine_predictions(
    predictions_and_outcomes,
    "doc_prediction",
    "algo_at_doc_prediction",
    "outcome",
    "algo_and_doc_prediction",
)
predictions_and_outcomes = analysis.combine_predictions(
    predictions_and_outcomes,
    "doc_prediction_cat",
    "algo_at_doc_prediction",
    "outcome",
    "algo_and_doc_cat_prediction",
)

## Discrimination and calibration of predictions

In [ ]:
table = analysis.performance_table(
    predictions=predictions_and_outcomes,
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model at Physician Prediction": "algo_at_doc_prediction",
        "Model at Patient Discharge": "algo_at_discharge",
        "Combined Model and Physician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and Physician (Category)": "algo_and_doc_cat_prediction",
    },
    curve="roc"
)
table.to_csv(os.path.join(project_root, tables_folder, hospital, "performance_table.csv"))
table

### Differences in model performance

In [ ]:
table, data = analysis.auc_differences_table(
    predictions=predictions_and_outcomes,
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction",
        "Combined Model and Physician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and Physician (Category)": "algo_and_doc_cat_prediction",
    },
    curve="roc"
)
table.to_csv(os.path.join(project_root, tables_folder, hospital, "auc_differences_table.csv"))
table

In [ ]:
plots.combined_auc_difference_plot(data)

In [ ]:
plots.plot_calibration(
    predictions=predictions_and_outcomes,
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Model": "algo_at_doc_prediction",
        "Combined Model and Physician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and Physician (Category)": "algo_and_doc_cat_prediction"
    }
)

## AUC with different time thresholds

In [ ]:
time_dif_data = analysis.create_max_time_difference_df(
    predictions_all_time_points=predictions_and_outcomes,
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction"
    },
    curve="roc"
)

In [ ]:
plots.plot_auc_at_max_time_difference(
    time_dif_data=time_dif_data,
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction"
    }
)

## Confidence level

In [ ]:
kruskal_wallis = kruskal(
    predictions_and_outcomes.loc[
        predictions_and_outcomes["confidence"] == "Veel", "confidence"
    ],
    predictions_and_outcomes.loc[
        predictions_and_outcomes["confidence"] == "Gemiddeld/Weinig", "confidence"
    ]
)
plots.stratified_histogram(
    predictions_and_outcomes,
    "doc_prediction",
    "confidence",
    kruskal=kruskal_wallis,
    labels=["High Confidence", "Medium/Low Confidence"],
    plotname="confidence_physician_prediction",
    title="Physician Predictions by self-reported Confidence"
)

In [ ]:
contingency_table_data = predictions_and_outcomes.loc[
    ~pd.isnull(predictions_and_outcomes.doc_prediction_cat), :
]
contingency_table = contingency.crosstab(
    contingency_table_data.doc_prediction_cat, contingency_table_data.confidence
).count
chi2 = chi2_contingency(contingency_table)
plots.barplot_confidence_cat_prediction(predictions_and_outcomes, "doc_prediction_cat", chi2)

In [ ]:
performance_table_high_confidence = analysis.performance_table(
    predictions=predictions_and_outcomes.loc[
        predictions_and_outcomes.confidence == "Veel", :
    ],
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction",
        "Combined Model and Physician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and Physician (Category)": "algo_and_doc_cat_prediction",
    },
    curve="roc"
)
performance_table_high_confidence.to_csv(os.path.join(project_root, tables_folder, hospital, "performance_table_high_confidence.csv"))

performance_table_medium_low_confidence = analysis.performance_table(
    predictions=predictions_and_outcomes.loc[
        predictions_and_outcomes.confidence == "Gemiddeld/Weinig", :
    ],
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction",
        "Combined Model and Physician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and Physician (Category)": "algo_and_doc_cat_prediction",
    },
    curve="roc"
)
performance_table_medium_low_confidence.to_csv(os.path.join(project_root, tables_folder, hospital, "performance_table_medium_low_confidence.csv"))

In [ ]:
performance_table_high_confidence

In [ ]:
performance_table_medium_low_confidence

In [ ]:
plots.plot_performance_by_confidence(
    performance_table_medium_low_confidence, 
    performance_table_high_confidence
    )

## Comparison factors for algo and doc

In [ ]:
# Define factors
shap_variable = predictions_and_outcomes.features_at_doc_prediction
factors_variable = predictions_and_outcomes.doc_factors
other_factor_variable = predictions_and_outcomes.doc_other_factor
patient_id_variable = predictions_and_outcomes.patient_id
discharge_time_variable = predictions_and_outcomes.discharge_time

# values are stored als 'stringified' lists. The construct_doc_factors_data_frame function assumes '||' as the separator
factors_variable1 = factors_variable.apply(
    lambda x: "||".join(eval(x)) if len(eval(x)) > 0 else None
)

other_factor_variable1 = other_factor_variable.apply(
    lambda x: "||".join(eval(x)) if len(eval(x)) > 0 else None
)

### Model factors (model features)

In [ ]:
# Get model factors based on Shapley values
shap_data = analysis.construct_shap_data_frame(
    shap_variable, patient_id_variable, discharge_time_variable
)

algo_factors = shap_data.pivot_table(
    index=["patient_id", "discharge_time"],
    columns=["feature_name"],
    values=["shap_value"],
    aggfunc=np.size
)

algo_factors_summary = pd.DataFrame((~algo_factors.isnull()).sum()).reset_index().rename(columns={
     0: 'count',
     'feature_name': 'factor'
     })[
    ['factor', 'count']].sort_values(
        by=['count', 'factor'], 
        ascending=[False, True]
        )

algo_factors_summary.to_csv(
    os.path.join(project_root, tables_folder, hospital, "algo_factors.csv"),
    index=False
    )

# Get model factors with new categories
shap_data_new = analysis.get_new_algo_features(shap_data, limit=3, absolute_values=True)
algo_factors_new = pd.pivot(
    shap_data_new,
    index=["patient_id", "discharge_time"],
    columns=["new_feature_name"],
    values=["shap_value"],
)

algo_factors_new_summary = pd.DataFrame((~algo_factors_new.isnull()).sum()).reset_index().rename(columns={
    0: 'count',
    'new_feature_name': 'factor_category'
    })[['factor_category', 'count']].sort_values(
        by=['count', 'factor_category'], 
        ascending=[False, True]
        )

algo_factors_new_summary['percentage'] = round(100 * algo_factors_new_summary['count']/algo_factors_new_summary['count'].sum(), 1)

algo_factors_new_summary.to_csv(
    os.path.join(project_root, tables_folder, hospital, "algo_factor_categories.csv"),
    index=False
    )

### Doc factors

In [ ]:
# Get physician reasons based on questionnaire answers
doc_factors_data = analysis.construct_doc_factors_data_frame(
    factors_variable1,
    other_factor_variable1,
    patient_id_variable,
    discharge_time_variable
)

doc_factors = doc_factors_data.pivot_table(
    index=["patient_id", "discharge_time"],
    columns=["factors"],
    values="factors",
    aggfunc=np.size
)

print(
    "Physicians on average give", (~doc_factors.isnull()).sum(axis=1).mean(), "reason"
)

doc_factors_summary = pd.DataFrame((~doc_factors.isnull()).sum()).reset_index().rename(columns={ 0: 'count', 'factors': 'factor'})[
    ['factor', 'count']].sort_values(
        by=['count', 'factor'], 
        ascending=[False, True]
        )

doc_factors_summary.to_csv(
    os.path.join(project_root, tables_folder, hospital, "doc_factors.csv"),
    index=False
    )

# Get physician reasons with new categories
doc_factors_data_new = analysis.get_new_doc_features(doc_factors_data)
doc_factors_new = pd.pivot(
    doc_factors_data_new,
    index=["patient_id", "discharge_time"],
    columns=["new_factors"],
    values="new_factors",
)

doc_factors_new_summary = pd.DataFrame((~doc_factors_new.isnull()).sum()).reset_index().rename(columns={ 
    0: 'count', 
    'new_factors': 'factor_category'
})[
    ['factor_category', 'count']].sort_values(
        by=['count', 'factor_category'], 
        ascending=[False, True]
        )

doc_factors_new_summary['percentage'] = round(100 * doc_factors_new_summary['count']/doc_factors_new_summary['count'].sum(), 1)

doc_factors_new_summary.to_csv(
    os.path.join(project_root, tables_folder, hospital, "doc_factor_categories.csv"),
    index=False
    )

### Model feature coverage
Calculates feature coverage based on the sum of absolute or raw Shapley values

In [ ]:
coverage_data = analysis.create_shap_coverage_data(shap_data, doc_factors_data_new)
plots.plot_coverage_at_max_features(coverage_data)

In [ ]:
plots.plot_combined_curves(
    predictions=predictions_and_outcomes,
    outcome="outcome",
    prediction_columns={
        "Physician (Percentage)": "doc_prediction",
        "Physician (Category)": "doc_prediction_cat",
        "Model": "algo_at_doc_prediction",
        "Combined Model and\nPhysician (Percentage)": "algo_and_doc_prediction",
        "Combined Model and\nPhysician (Category)": "algo_and_doc_cat_prediction",
    },
    curve="roc"
)